In [42]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os

In [43]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

## Data Loading

### Writing PyG Dataset

In [3]:
import torch

In [4]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [5]:
from torch_geometric.utils import from_networkx, to_networkx

In [6]:
def clustering_coefficient(G, node):
    ns = [n for n in G.neighbors(node)]
    if len(ns) <= 1:
        return 0
    
    numerator = 0
    denominator = len(ns) * (len(ns) - 1) / 2
    for i in range(0, len(ns)):
        for j in range(i+1, len(ns)):
            n1, n2 = ns[i], ns[j]
            numerator += G.has_edge(n1, n2)
    
    return numerator / denominator
                

In [7]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 2)
    ind = 0
    for node in G.nodes():
        # set node degree as feature
        x[ind][0] = G.degree[node]
        x[ind][1] = clustering_coefficient(G, node)
        ind += 1
    return x

In [8]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        for ind, graph in enumerate(total_laman_data[0]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        for ind, graph in enumerate(total_laman_data[1]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [9]:
DATA_PATH = "../data-2d/data/4096-20-4-entries-med.pkl.gz"

In [10]:
laman_data = LamanDataset("", DATA_PATH)

In [11]:
laman_data[0]

Data(edge_index=[2, 122], x=[32, 2], label=[1], num_nodes=32)

## Split into Train / Test

In [12]:
from torch.utils.data import random_split

proportions = [.6, .4]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [13]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 256, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 256, shuffle=True)

In [14]:
print("Number of train batches: ", len(train_loader))
print("Number of test batches: ", len(test_loader))

Number of train batches:  10
Number of test batches:  7


In [15]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 19192], x=[5157, 2], label=[256], num_nodes=5157, batch=[5157], ptr=[257])


## Model Architecture & Training

In [22]:
from gin.gin import GIN

In [23]:
model = GIN(num_features=1)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [24]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.001)

# scheduler = ReduceLROnPlateau(optimizer, 'min', min_lr=1e-6, verbose=True, patience=10)

In [29]:
def train(model, data, features_to_use):
    ind = 0
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, model

In [30]:
def check_accuracy(model, loader, features_to_use):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)*100

In [32]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, model = train(model, train_loader, [0])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader, [0]), check_accuracy(model, test_loader, [0])
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...
Epoch 0 | Train loss 0.30869102478027344
Train Accuracy 90.72039072039072 | Test Accuracy 90.42098840756559
Epoch 1 | Train loss 0.2955162227153778
Train Accuracy 91.29019129019129 | Test Accuracy 90.9090909090909
Epoch 2 | Train loss 0.23613198101520538
Train Accuracy 91.85999185999187 | Test Accuracy 91.2751677852349
Epoch 3 | Train loss 0.22741472721099854
Train Accuracy 91.41229141229141 | Test Accuracy 91.15314215985357
Epoch 4 | Train loss 0.33309268951416016
Train Accuracy 91.57509157509158 | Test Accuracy 91.2751677852349
Epoch 5 | Train loss 0.26165688037872314
Train Accuracy 91.77859177859177 | Test Accuracy 91.33618059792556
Epoch 6 | Train loss 0.21904051303863525
Train Accuracy 92.22629222629223 | Test Accuracy 91.70225747406955
Epoch 7 | Train loss 0.2693001925945282
Train Accuracy 92.22629222629223 | Test Accuracy 91.94630872483222
Epoch 8 | Train loss 0.2384323924779892
Train Accuracy 92.10419210419211 | Test Accuracy 91.70225747406955
Epoch 9 | Tra

Epoch 76 | Train loss 0.12972289323806763
Train Accuracy 98.94179894179894 | Test Accuracy 98.90176937156802
Epoch 77 | Train loss 0.14269527792930603
Train Accuracy 98.77899877899878 | Test Accuracy 98.65771812080537
Epoch 78 | Train loss 0.11841492354869843
Train Accuracy 98.81969881969881 | Test Accuracy 98.7797437461867
Epoch 79 | Train loss 0.12571583688259125
Train Accuracy 98.86039886039886 | Test Accuracy 98.7797437461867
Epoch 80 | Train loss 0.12559713423252106
Train Accuracy 98.9010989010989 | Test Accuracy 98.96278218425869
Epoch 81 | Train loss 0.10437126457691193
Train Accuracy 98.65689865689866 | Test Accuracy 98.53569249542404
Epoch 82 | Train loss 0.12889762222766876
Train Accuracy 98.9010989010989 | Test Accuracy 98.7797437461867
Epoch 83 | Train loss 0.10427843779325485
Train Accuracy 99.1045991045991 | Test Accuracy 99.02379499694935
Epoch 84 | Train loss 0.0913592204451561
Train Accuracy 98.61619861619862 | Test Accuracy 98.53569249542404
Epoch 85 | Train loss 0.11

Epoch 151 | Train loss 0.07229060679674149
Train Accuracy 99.71509971509973 | Test Accuracy 99.20683343502135
Epoch 152 | Train loss 0.06120862439274788
Train Accuracy 99.63369963369964 | Test Accuracy 99.20683343502135
Epoch 153 | Train loss 0.06579317152500153
Train Accuracy 99.67439967439967 | Test Accuracy 99.32885906040269
Epoch 154 | Train loss 0.05331505089998245
Train Accuracy 99.47089947089947 | Test Accuracy 99.26784624771201
Epoch 155 | Train loss 0.06411831080913544
Train Accuracy 99.67439967439967 | Test Accuracy 99.32885906040269
Epoch 156 | Train loss 0.060359612107276917
Train Accuracy 99.43019943019942 | Test Accuracy 99.26784624771201
Epoch 157 | Train loss 0.07096514105796814
Train Accuracy 99.43019943019942 | Test Accuracy 99.02379499694935
Epoch 158 | Train loss 0.052890025079250336
Train Accuracy 99.55229955229954 | Test Accuracy 99.38987187309336
Epoch 159 | Train loss 0.0666571706533432
Train Accuracy 99.67439967439967 | Test Accuracy 99.32885906040269
Epoch 160

Epoch 226 | Train loss 0.032711323350667953
Train Accuracy 99.71509971509973 | Test Accuracy 99.32885906040269
Epoch 227 | Train loss 0.02888377755880356
Train Accuracy 99.75579975579976 | Test Accuracy 99.38987187309336
Epoch 228 | Train loss 0.03092672862112522
Train Accuracy 99.51159951159951 | Test Accuracy 99.45088468578402
Epoch 229 | Train loss 0.043851338326931
Train Accuracy 99.5929995929996 | Test Accuracy 99.38987187309336
Epoch 230 | Train loss 0.03599230572581291
Train Accuracy 99.63369963369964 | Test Accuracy 99.38987187309336
Epoch 231 | Train loss 0.024900326505303383
Train Accuracy 99.55229955229954 | Test Accuracy 99.38987187309336
Epoch 232 | Train loss 0.026323162019252777
Train Accuracy 99.79649979649979 | Test Accuracy 99.38987187309336
Epoch 233 | Train loss 0.03634965047240257
Train Accuracy 99.75579975579976 | Test Accuracy 99.38987187309336
Epoch 234 | Train loss 0.029081933200359344
Train Accuracy 99.71509971509973 | Test Accuracy 99.45088468578402
Epoch 235

Train Accuracy 99.75579975579976 | Test Accuracy 99.51189749847468
Epoch 301 | Train loss 0.031475506722927094
Train Accuracy 99.79649979649979 | Test Accuracy 99.38987187309336
Epoch 302 | Train loss 0.024512210860848427
Train Accuracy 99.71509971509973 | Test Accuracy 99.45088468578402
Epoch 303 | Train loss 0.014314848929643631
Train Accuracy 99.79649979649979 | Test Accuracy 99.38987187309336
Epoch 304 | Train loss 0.025660421699285507
Train Accuracy 99.75579975579976 | Test Accuracy 99.57291031116534
Epoch 305 | Train loss 0.025018155574798584
Train Accuracy 99.79649979649979 | Test Accuracy 99.51189749847468
Epoch 306 | Train loss 0.02141355350613594
Train Accuracy 99.75579975579976 | Test Accuracy 99.51189749847468
Epoch 307 | Train loss 0.022104019299149513
Train Accuracy 99.79649979649979 | Test Accuracy 99.51189749847468
Epoch 308 | Train loss 0.013795880600810051
Train Accuracy 99.71509971509973 | Test Accuracy 99.51189749847468
Epoch 309 | Train loss 0.024647759273648262
Tr

Epoch 375 | Train loss 0.01557453814893961
Train Accuracy 99.22669922669922 | Test Accuracy 99.08480780964003
Epoch 376 | Train loss 0.020806021988391876
Train Accuracy 99.71509971509973 | Test Accuracy 99.51189749847468
Epoch 377 | Train loss 0.02331693470478058
Train Accuracy 99.75579975579976 | Test Accuracy 99.51189749847468
Epoch 378 | Train loss 0.02701166830956936
Train Accuracy 99.79649979649979 | Test Accuracy 99.57291031116534
Epoch 379 | Train loss 0.02102089114487171
Train Accuracy 99.79649979649979 | Test Accuracy 99.45088468578402
Epoch 380 | Train loss 0.011618003249168396
Train Accuracy 99.83719983719985 | Test Accuracy 99.51189749847468
Epoch 381 | Train loss 0.016607100144028664
Train Accuracy 99.71509971509973 | Test Accuracy 99.51189749847468
Epoch 382 | Train loss 0.010777733288705349
Train Accuracy 99.71509971509973 | Test Accuracy 99.57291031116534
Epoch 383 | Train loss 0.018506767228245735
Train Accuracy 99.71509971509973 | Test Accuracy 99.45088468578402
Epoch

Epoch 449 | Train loss 0.012812490575015545
Train Accuracy 99.79649979649979 | Test Accuracy 99.57291031116534
Epoch 450 | Train loss 0.016105523332953453
Train Accuracy 99.83719983719985 | Test Accuracy 99.51189749847468
Epoch 451 | Train loss 0.010558735579252243
Train Accuracy 99.71509971509973 | Test Accuracy 99.45088468578402
Epoch 452 | Train loss 0.027034837752580643
Train Accuracy 99.22669922669922 | Test Accuracy 99.14582062233069
Epoch 453 | Train loss 0.022710712626576424
Train Accuracy 99.87789987789988 | Test Accuracy 99.45088468578402
Epoch 454 | Train loss 0.012491337954998016
Train Accuracy 99.87789987789988 | Test Accuracy 99.45088468578402
Epoch 455 | Train loss 0.008951723575592041
Train Accuracy 99.3080993080993 | Test Accuracy 99.14582062233069
Epoch 456 | Train loss 0.009639565832912922
Train Accuracy 99.71509971509973 | Test Accuracy 99.51189749847468
Epoch 457 | Train loss 0.010153346695005894
Train Accuracy 99.79649979649979 | Test Accuracy 99.57291031116534
Ep

KeyboardInterrupt: 

In [ ]:
# sanity check

In [ ]:
# test on best model

In [33]:
# square
import networkx as nx
square = nx.Graph()
square.add_edge(0, 1)
square.add_edge(1, 3)
square.add_edge(0, 2)
square.add_edge(2, 3)

In [34]:
# square with cross bar (rigid)
import networkx as nx
square_bar = nx.Graph()
square_bar.add_edge(0, 1)
square_bar.add_edge(1, 3)
square_bar.add_edge(0, 2)
square_bar.add_edge(2, 3)
square_bar.add_edge(0, 3)

In [35]:
# triangle
import networkx as nx
triangle = nx.Graph()
triangle.add_edge(0, 1)
triangle.add_edge(0, 2)
triangle.add_edge(1, 2)

In [36]:
# pentagon
import networkx as nx
pentagon = nx.Graph()
pentagon.add_edge(0, 1)
pentagon.add_edge(1, 3)
pentagon.add_edge(3, 4)
pentagon.add_edge(4, 2)
pentagon.add_edge(2, 0)

In [41]:
toy_problems = [square, square_bar, triangle, pentagon]
labels = [1, 0, 0, 1]

for index, toy_problem in enumerate(toy_problems):
    graph_as_data = from_networkx(toy_problem)
    graph_as_data.x = generate_feature_vector(toy_problem)
    graph_as_data.label = labels[index]
    validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
    for batch in validation_set:
        pred = bestModel(batch.x[:, [0]], batch.edge_index, batch.batch)
        print(pred[0])
    

tensor([[1.]], grad_fn=<SigmoidBackward0>)
tensor([[0.8556]], grad_fn=<SigmoidBackward0>)
tensor([[0.9999]], grad_fn=<SigmoidBackward0>)
tensor([[1.]], grad_fn=<SigmoidBackward0>)


In [41]:
graph_as_data

Data(edge_index=[2, 10], num_nodes=5, x=[5, 2], label=1)

In [42]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [43]:
for batch in validation_set:
    pred = bestModel(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])

tensor([[1.]], grad_fn=<SigmoidBackward0>)
tensor([[41.9681]], grad_fn=<AddmmBackward0>)


In [44]:
class LamanTestDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_test.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 1)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 64)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [45]:
# add functionality to support a test dataset
TEST_DATA_PATH = "../data-2d/data/test-dataset-30loc-5std.pkl.gz"
laman_test_set = LamanTestDataset("", TEST_DATA_PATH)

In [46]:
from torch_geometric.loader import DataLoader
laman_test_loader = DataLoader(laman_test_set, batch_size = 2, shuffle=True)

In [47]:
random_test_acc = check_accuracy(bestModel, laman_test_loader)
print(f"Accuracy {random_test_acc}")

Accuracy 96.875


In [ ]:
# generate statistics on the data

In [48]:
# test the clustering coefficient

In [50]:
clustering_coefficient(square, 0)

0.0

In [51]:
clustering_coefficient(triangle, 0)

1.0

In [52]:
clustering_coefficient(square_bar, 0)

0.6666666666666666

In [54]:
# generate graph correlating clustering coefficient to rigidity

In [59]:
for item in train_data:
    item = to_networkx(item)
    print(type(item))
    
    break

<class 'networkx.classes.digraph.DiGraph'>


In [ ]:
torch_geometric.utils.convert.to_networkx()

# Scratch Work: Sahil

In [60]:
total_laman_data = None
with gzip.open(DATA_PATH, 'r') as f:
    total_laman_data = pickle.load(f)

In [61]:
sample_graph = total_laman_data[0][0]

In [63]:
print(type(sample_graph))

<class 'networkx.classes.graph.Graph'>


In [67]:
to_data = from_networkx(sample_graph)
from_data = to_networkx(to_data, to_undirected = True)

In [68]:
print(type(from_data))

<class 'networkx.classes.graph.Graph'>


In [80]:
def compute_min_clustering_coefficient(G):
    min_coefficient = 1
    for node in G.nodes():
        min_coefficient = min(min_coefficient, clustering_coefficient(G, node))
        
    return min_coefficient

In [82]:
for index, sample_graph in enumerate(train_data):
    label = sample_graph.label
    networkx_sample_graph = to_networkx(sample_graph, to_undirected = True)
    print(label, " ", index, " ", compute_min_clustering_coefficient(networkx_sample_graph))
    
    if index == 10:
        break

tensor([1])   0   0.0
tensor([1])   1   0.0
tensor([0])   2   0.0
tensor([0])   3   0.0
tensor([0])   4   0.0
tensor([0])   5   0.0
tensor([1])   6   0.0
tensor([0])   7   0.0
tensor([0])   8   0.0
tensor([1])   9   0.0
tensor([0])   10   0.0


In [81]:
print(compute_min_clustering_coefficient(from_data))

0.0


In [83]:
compute_min_clustering_coefficient(triangle)

1

In [ ]:
# what if instead of training a gnn – you just trained on 

In [84]:
# train a binary classifier on just the degrees of the nodes



In [85]:
# train a binary classifier on just the triangle feature

In [86]:
# train a binary classifier on the triangle features and the degree

In [87]:
# train a network with just degree of the node 

In [91]:
model_just_degree = GIN(num_features=1)
print(model_just_degree)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=1, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): ReLU()
  ))
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True,

In [96]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, h = train(train_loader, [0])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model_just_degree, train_loader), check_accuracy(model_just_degree, test_loader)
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...


IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 0] (got interval [0, 5138])

In [97]:
generate_feature_vector(sample_graph)

AttributeError: 'GlobalStorage' object has no attribute 'number_of_nodes'

In [110]:
sample_graph.x[:, [0, 1]]

tensor([[5.0000, 0.3000],
        [7.0000, 0.1429],
        [3.0000, 0.0000],
        [5.0000, 0.0000],
        [6.0000, 0.1333],
        [3.0000, 0.0000],
        [5.0000, 0.3000],
        [6.0000, 0.1333],
        [3.0000, 0.3333],
        [2.0000, 0.0000],
        [3.0000, 0.3333],
        [5.0000, 0.2000],
        [3.0000, 0.0000],
        [2.0000, 0.0000],
        [2.0000, 1.0000],
        [3.0000, 0.6667],
        [3.0000, 0.3333]])